In [14]:
import graphlab

In [15]:
loans = graphlab.SFrame('../lending-club-data.gl/')

In [16]:
loans.head(5)

Columns:
	id	int
	member_id	int
	loan_amnt	int
	funded_amnt	int
	funded_amnt_inv	int
	term	str
	int_rate	float
	installment	float
	grade	str
	sub_grade	str
	emp_title	str
	emp_length	str
	home_ownership	str
	annual_inc	int
	is_inc_v	str
	issue_d	str
	loan_status	str
	pymnt_plan	str
	url	str
	desc	str
	purpose	str
	title	str
	zip_code	str
	addr_state	str
	dti	float
	delinq_2yrs	int
	earliest_cr_line	str
	inq_last_6mths	int
	mths_since_last_delinq	int
	mths_since_last_record	int
	open_acc	int
	pub_rec	int
	revol_bal	int
	revol_util	float
	total_acc	int
	initial_list_status	str
	out_prncp	float
	out_prncp_inv	float
	total_pymnt	float
	total_pymnt_inv	float
	total_rec_prncp	float
	total_rec_int	float
	total_rec_late_fee	float
	recoveries	float
	collection_recovery_fee	float
	last_pymnt_d	str
	last_pymnt_amnt	float
	next_pymnt_d	str
	last_credit_pull_d	str
	collections_12_mths_ex_med	int
	mths_since_last_major_derog	str
	policy_code	int
	not_compliant	int
	status	str
	inactive_loans	int
	bad_loans	int
	emp_length_num	int
	grade_num	int
	sub_grade_num	float
	delinq_2yrs_zero	int
	pub_rec_zero	int
	collections_12_mths_zero	int
	short_emp	int
	payment_inc_ratio	float
	final_d	str
	last_delinq_none	int
	last_record_none	int
	last_major_derog_none	int

Rows: 5

Data:
+---------+-----------+-----------+-------------+-----------------+------------+
|    id   | member_id | loan_amnt | funded_amnt | funded_amnt_inv |    term    |
+---------+-----------+-----------+-------------+-----------------+------------+
| 1077501 |  1296599  |    5000   |     5000    |       4975      |  36 months |
| 1077430 |  1314167  |    2500   |     2500    |       2500      |  60 months |
| 1077175 |  1313524  |    2400   |     2400    |       2400      |  36 months |
| 1076863 |  1277178  |   10000   |    10000    |      10000      |  36 months |
| 1075269 |  1311441  |    5000   |     5000    |       5000      |  36 months |
+---------+-----------+-----------+-------------+-----------------+------------+
+----------+-------------+-------+-----------+----------------------+------------+
| int_rate | installment | grade | sub_grade |      emp_title       | emp_length |
+----------+-------------+-------+-----------+----------------------+------------+
|  10.65   |    162.87   |   B   |     B2    |                      | 10+ years  |
|  15.27   |    59.83    |   C   |     C4    |        Ryder         |  < 1 year  |
|  15.96   |    84.33    |   C   |     C5    |                      | 10+ years  |
|  13.49   |    339.31   |   C   |     C1    | AIR RESOURCES BOARD  | 10+ years  |
|   7.9    |    156.46   |   A   |     A4    | Veolia Transportaton |  3 years   |
+----------+-------------+-------+-----------+----------------------+------------+
+----------------+------------+-----------------+-----------------+-------------+
| home_ownership | annual_inc |     is_inc_v    |     issue_d     | loan_status |
+----------------+------------+-----------------+-----------------+-------------+
|      RENT      |   24000    |     Verified    | 20111201T000000 |  Fully Paid |
|      RENT      |   30000    | Source Verified | 20111201T000000 | Charged Off |
|      RENT      |   12252    |   Not Verified  | 20111201T000000 |  Fully Paid |
|      RENT      |   49200    | Source Verified | 20111201T000000 |  Fully Paid |
|      RENT      |   36000    | Source Verified | 20111201T000000 |  Fully Paid |
+----------------+------------+-----------------+-----------------+-------------+
+------------+-------------------------------+-------------------------------+-----+
| pymnt_plan |              url              |              desc             | ... |
+------------+-------------------------------+-------------------------------+-----+
|     n      | https://www.lendingclub.co... |   Borrower added on 12/22/... | ... |
|     n      | https://www.lendingclub.co... |   Borrower added on 12/22/... | ... |
|     n      | https://www.lendingclub.co... |                               | ... 

In [17]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x: +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

In [18]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [19]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


In [20]:
loans_data = risky_loans.append(safe_loans)
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

In [21]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
features

['grade.A',
 'grade.B',
 'grade.C',
 'grade.D',
 'grade.E',
 'grade.F',
 'grade.G',
 'term. 36 months',
 'term. 60 months',
 'home_ownership.MORTGAGE',
 'home_ownership.OTHER',
 'home_ownership.OWN',
 'home_ownership.RENT',
 'emp_length.1 year',
 'emp_length.10+ years',
 'emp_length.2 years',
 'emp_length.3 years',
 'emp_length.4 years',
 'emp_length.5 years',
 'emp_length.6 years',
 'emp_length.7 years',
 'emp_length.8 years',
 'emp_length.9 years',
 'emp_length.< 1 year',
 'emp_length.n/a']

In [22]:
print "Number of features (after binarizing categorical variables) = %s" % len(features)

Number of features (after binarizing categorical variables) = 25


In [29]:
loans_data.tail(5)

Columns:
	safe_loans	int
	grade.A	int
	grade.B	int
	grade.C	int
	grade.D	int
	grade.E	int
	grade.F	int
	grade.G	int
	term. 36 months	int
	term. 60 months	int
	home_ownership.MORTGAGE	int
	home_ownership.OTHER	int
	home_ownership.OWN	int
	home_ownership.RENT	int
	emp_length.1 year	int
	emp_length.10+ years	int
	emp_length.2 years	int
	emp_length.3 years	int
	emp_length.4 years	int
	emp_length.5 years	int
	emp_length.6 years	int
	emp_length.7 years	int
	emp_length.8 years	int
	emp_length.9 years	int
	emp_length.< 1 year	int
	emp_length.n/a	int

Rows: 5

Data:
+------------+---------+---------+---------+---------+---------+---------+---------+
| safe_loans | grade.A | grade.B | grade.C | grade.D | grade.E | grade.F | grade.G |
+------------+---------+---------+---------+---------+---------+---------+---------+
|     1      |    0    |    1    |    0    |    0    |    0    |    0    |    0    |
|     1      |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
|     1      |    1    |    0    |    0    |    0    |    0    |    0    |    0    |
|     1      |    0    |    0    |    0    |    0    |    0    |    0    |    1    |
|     1      |    0    |    0    |    0    |    1    |    0    |    0    |    0    |
+------------+---------+---------+---------+---------+---------+---------+---------+
+-----------------+-----------------+-------------------------+----------------------+
| term. 36 months | term. 60 months | home_ownership.MORTGAGE | home_ownership.OTHER |
+-----------------+-----------------+-------------------------+----------------------+
|        1        |        0        |            1            |          0           |
|        1        |        0        |            1            |          0           |
|        1        |        0        |            0            |          0           |
|        0        |        1        |            1            |          0           |
|        1        |        0        |            1            |          0           |
+-----------------+-----------------+-------------------------+----------------------+
+--------------------+---------------------+-------------------+----------------------+
| home_ownership.OWN | home_ownership.RENT | emp_length.1 year | emp_length.10+ years |
+--------------------+---------------------+-------------------+----------------------+
|         0          |          0          |         0         |          0           |
|         0          |          0          |         0         |          0           |
|         1          |          0          |         0         |          0           |
|         0          |          0          |         0         |          1           |
|         0          |          0          |         0         |          1           |
+--------------------+---------------------+-------------------+----------------------+
+--------------------+--------------------+--------------------+--------------------+-----+
| emp_length.2 years | emp_length.3 years | emp_length.4 years | emp_length.5 years | ... |
+--------------------+--------------------+--------------------+--------------------+-----+
|         0          |         0          |         0          |         0          | ... |
|         0          |         0          |         0          |         1          | ... |
|         0          |         1          |         0          |         0          | ... |
|         0          |         0          |         0          |         0          | ... |
|         0          |         0          |         0          |         0          | ... |
+--------------------+--------------------+--------------------+--------------------+-----+
[5 rows x 26 columns]

In [24]:
print "Total number of grade.A loans : %s" % loans_data['grade.A'].sum()
print "Expexted answer               : 6422"

Total number of grade.A loans : 6422
Expexted answer               : 6422


In [25]:
train_data, test_data = loans_data.random_split(.8, seed=1)

In [57]:
def intermediate_node_num_mistakes(labels_in_node):
    # Corner case: If labels_in_node is empty, return 0
    if len(labels_in_node) == 0:
        return 0
    
    # Count the number of 1's (safe loans)
    ## YOUR CODE HERE
    safe_loans = labels_in_node==1
    safe_loans = safe_loans.sum()
    # Count the number of -1's (risky loans)
    ## YOUR CODE HERE
    risky_loans = labels_in_node==-1
    risky_loans = risky_loans.sum()
    # Return the number of mistakes that the majority classifier makes.
    ## YOUR CODE HERE
    if safe_loans>risky_loans:
        mistakes = risky_loans
    else:
        mistakes = safe_loans
    return mistakes

In [58]:
# Test case 1
example_labels = graphlab.SArray([-1, -1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 1 failed... try again!'

# Test case 2
example_labels = graphlab.SArray([-1, -1, 1, 1, 1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 2 failed... try again!'
    
# Test case 3
example_labels = graphlab.SArray([-1, -1, -1, -1, -1, 1, 1])
if intermediate_node_num_mistakes(example_labels) == 2:
    print 'Test passed!'
else:
    print 'Test 3 failed... try again!'

Test passed!
Test passed!
Test passed!


In [55]:
def best_splitting_feature(data, features, target):
    
    best_feature = None # Keep track of the best feature 
    best_error = 10     # Keep track of the best error so far 
    # Note: Since error is always <= 1, we should intialize it with something larger than 1.

    # Convert to float to make sure error gets computed correctly.
    num_data_points = float(len(data))  
    
    # Loop through each feature to consider splitting on that feature
    for feature in features:
        
        # The left split will have all data points where the feature value is 0
        left_split = data[data[feature] == 0]
        
        # The right split will have all data points where the feature value is 1
        ## YOUR CODE HERE
        right_split =  data[data[feature] == 1]
            
        # Calculate the number of misclassified examples in the left split.
        # Remember that we implemented a function for this! (It was called intermediate_node_num_mistakes)
        # YOUR CODE HERE
        left_mistakes = intermediate_node_num_mistakes(left_split[target])
        
        # Calculate the number of misclassified examples in the right split.
        ## YOUR CODE HERE
        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        # Compute the classification error of this split.
        # Error = (# of mistakes (left) + # of mistakes (right)) / (# of data points)
        ## YOUR CODE HERE
       
        error = float((left_mistakes+right_mistakes)/num_data_points)

        # If this is the best error we have found so far, store the feature as best_feature and the error as best_error
        ## YOUR CODE HERE
        if error < best_error:
            best_error = error
            best_feature = feature
    
    return best_feature # Return the best feature we found

In [59]:
if best_splitting_feature(train_data, features, 'safe_loans') == 'term. 36 months':
    print 'Test passed!'
else:
    print 'Test failed... try again!'

Test passed!


In [73]:
def create_leaf(target_values):
    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf':  True   }   ## YOUR CODE HERE
    
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])
    
    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1       ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1      ## YOUR CODE HERE
        
    # Return the leaf node        
    return leaf 

In [79]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    remaining_features = features[:] # Make a copy of the features.
    
    target_values = data[target]
    print "--------------------------------------------------------------------"
    print "Subtree, depth = %s (%s data points)." % (current_depth, len(target_values))
    

    # Stopping condition 1
    # (Check if there are mistakes at current node.
    # Recall you wrote a function intermediate_node_num_mistakes to compute this.)
    if intermediate_node_num_mistakes(target_values)== 0:  ## YOUR CODE HERE
        print "Stopping condition 1 reached."     
        # If not mistakes at current node, make current node a leaf node
        return create_leaf(target_values)
    
    # Stopping condition 2 (check if there are remaining features to consider splitting on)
    if remaining_features == []:   ## YOUR CODE HERE
        print "Stopping condition 2 reached."    
        # If there are no remaining features to consider, make current node a leaf node
        return create_leaf(target_values)    
    
    # Additional stopping condition (limit tree depth)
    if current_depth >= max_depth :  ## YOUR CODE HERE
        print "Reached maximum depth. Stopping for now."
        # If the max tree depth has been reached, make current node a leaf node
        return create_leaf(target_values)

    # Find the best splitting feature (recall the function best_splitting_feature implemented above)
    ## YOUR CODE HERE
    
    splitting_feature = best_splitting_feature(data, features, target)
    
    # Split on the best feature that we found. 
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]      ## YOUR CODE HERE
    remaining_features.remove(splitting_feature)
    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target])
        ## YOUR CODE HERE

        
    # Repeat (recurse) on left and right subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    ## YOUR CODE HERE
    right_tree =  decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

In [77]:
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [80]:
small_data_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 3)
if count_nodes(small_data_decision_tree) == 13:
    print 'Test passed!'
else:
    print 'Test failed... try again!'
    print 'Number of nodes found                :', count_nodes(small_data_decision_tree)
    print 'Number of nodes that should be there : 13' 

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 3 (1048 data points).
Reached maximum depth. Stopping for now.
--------------------------------------------------------------------
Subtree, depth = 2 (101 data points).
Split on feature emp_length.n/a. (96, 5)
--------------------------------------------------------------------
Subtree, depth = 3 (96 data points)

In [81]:
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth=6)

--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term. 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade.A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade.B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade.C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade.D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade.E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points).
R

In [82]:
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [83]:
test_data[0]

{'emp_length.1 year': 0,
 'emp_length.10+ years': 0,
 'emp_length.2 years': 1,
 'emp_length.3 years': 0,
 'emp_length.4 years': 0,
 'emp_length.5 years': 0,
 'emp_length.6 years': 0,
 'emp_length.7 years': 0,
 'emp_length.8 years': 0,
 'emp_length.9 years': 0,
 'emp_length.< 1 year': 0,
 'emp_length.n/a': 0,
 'grade.A': 0,
 'grade.B': 0,
 'grade.C': 0,
 'grade.D': 1,
 'grade.E': 0,
 'grade.F': 0,
 'grade.G': 0,
 'home_ownership.MORTGAGE': 0,
 'home_ownership.OTHER': 0,
 'home_ownership.OWN': 0,
 'home_ownership.RENT': 1,
 'safe_loans': -1,
 'term. 36 months': 0,
 'term. 60 months': 1}

In [84]:
print 'Predicted class: %s ' % classify(my_decision_tree, test_data[0])

Predicted class: -1 


In [85]:
classify(my_decision_tree, test_data[0], annotate=True)

Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
At leaf, predicting -1


-1

In [86]:
def evaluate_classification_error(tree, data, target):
    # Apply the classify(tree, x) to each row in your data
    predictions = data.apply(lambda x: classify(tree, x))
    
    # Once you've made the predictions, calculate the classification error and return it
    mistakes = sum(predictions != data[target])
    return float(mistakes)/len(data)

In [87]:

round(evaluate_classification_error(my_decision_tree, test_data, 'safe_loans'), 2)

0.38

In [88]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print "(leaf, label: %s)" % tree['prediction']
        return None
    split_feature, split_value = split_name.split('.')
    print '                       %s' % name
    print '         |---------------|----------------|'
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '  [{0} == 0]               [{0} == 1]    '.format(split_name)
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree'))

In [89]:
print_stump(my_decision_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [90]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [91]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade.A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.B == 0]               [grade.B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [92]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.D == 0]               [grade.D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [93]:
print_stump(my_decision_tree['right']['right'], my_decision_tree['right']['splitting_feature'])

(leaf, label: -1)
